# Grover's Algorithm and the Amplitude Amplification

Grover's algorithm is one of the most famous quantum algorithm. Grover's algorithm is a very powerful and useful algorithm in a wide range of fiealds. For the details of Grover's algorithm, please see [Grover's Algorithm](https://qiskit.org/textbook/ch-algorithms/grover.html) in the Qiskit textbook.

Qiskit Aqua has a special class named `Grover` for Grover's algorithm. `Grover` can specify various problem configurations with its parameters such as oracles, state preparation circuits, the number of the iterations, and so on.

## Grover's algorithm
### Running Grover's algorithm with the `Grover` class
To run Grover's algorithm with the `Grover` class, firstly, we need to specify an oracle for the circuit of Grover's algorithm. In the following example, we use `QuantumCircuit` as the oracle of Grover's algorithm. However, there are several other class that we can use as the oracle of Grover's algorithm. We talk about them laster in this tutorial. To specify the oracle, we just simply pass `QuantumCircuit` for the oracle. Note that the oracle for `Grover` mush be one that changes the phase of "good state" (those state are called as `good_state`) in `Grover`. If you'd like to use an ancilla bit and a CNOT gate to change the phase of "good state", please don't forget to put an X gate and an Hadamard gate onto the target bit of the CNOT gate.



In [1]:
from qiskit import QuantumCircuit
from qiskit.aqua.algorithms import Grover

# `QuantumCircuit` as the `oracle` argument
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=['11'])
# oracle
print(grover._grover_operator.oracle)

        
q_0: ─■─
      │ 
q_1: ─■─
        


In [2]:
# Grover operator
print(grover._grover_operator)

            ┌───┐┌───┐          ┌───┐┌───┐     
state_0: ─■─┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
          │ ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐
state_1: ─■─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
            └───┘└───┘└───┘└───┘└───┘└───┘└───┘


Then, we specify a backend and call the `run` method of `Grover` with the backend. The result of `run` will be `GroverResult`. `oracle_evaluation` of `GroverResult` will be `True` if `top_measurment` is one of "good state". Otherwise, `oracle_evaluation` will be False.


In [3]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance

qasm_simulator = QuantumInstance(BasicAer.get_backend('qasm_simulator'))
result = grover.run(quantum_instance=qasm_simulator)
print(type(result))
print(result)


<class 'qiskit.aqua.algorithms.amplitude_amplifiers.grover.GroverResult'>
{'measurement': {'11': 1024}, 'top_measurement': '11', 'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x106337c70>, 'assignment': '11', 'oracle_evaluation': True}


In the example, the result of `top_measurement` is `11` which is one of "good state". Thus, we succeeded to find the answer by using `Grover`.

### Using the different types of classes as the oracle of `Grover`
In the above example, we used `QuantumCircuit` as the oracle of `Grover`. 
We can also use `Oracle`, and `StateVector` as `oracle`.
All the following examples are when $|11\rangle$ is "good state"

Note that `Oracle` has 3 subclasses. They are `LogicalExpressionOracle`, `TrutheTableOracle`, and `CustomCircuitOracle`. Each of them aims for different usecases. For example, `LogicalExpressionOracle` is expecially useful for solving 3-SAT problems (see [Using Grover's Algorithm to Perform Quantum Search](https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/optimization/grover.ipynb) in Qiskit-community-tutorials).


Note that, in the following `LogicalExpressionOracle` example, `LogicalExpressionOracle` uses an ancilla bit to flip the phase of the good state by using the phase kickback ([Phase Kickback](https://qiskit.org/textbook/ch-gates/phase-kickback.html)). You don't need to pass `good_state` in `Grover` since `Oracle` itsself has `evaluate_classically()` for checking whether an answer is correct or not.

In [4]:
from qiskit.aqua.components.oracles import LogicalExpressionOracle

# `Oracle` (`LogicalExpressionOracle`) as the `oracle` argument
expression = '(a & b)'
oracle = LogicalExpressionOracle(expression)
grover = Grover(oracle=oracle)
print(grover._grover_operator.oracle)

                              
q_0: ────────────■────────────
                 │            
q_1: ────────────■────────────
     ┌───┐┌───┐┌─┴─┐┌───┐┌───┐
q_2: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘└───┘└───┘


In [5]:
from qiskit.quantum_info import Statevector
oracle = Statevector.from_label('11')
grover = Grover(oracle=oracle, good_state=['11'])
print(grover._grover_operator.oracle)


     ┌─────────────┐              
q_0: ┤0            ├──────────────
     │  UCRZ(0,pi) │┌────────────┐
q_1: ┤1            ├┤ UCRZ(pi/2) ├
     └─────────────┘└────────────┘


## Amplitude amplification
Grover's algorithm uses Hadamard gates to create the uniform superposition of all the states at the beginning of the grover circuit (called a `state_preparation` circuit in `Grover`). We can use other gates to create the uniform superposition of specific states and we can also use them instead of those Hadamard gates in usual grover circuits. By using the uniform superposition of specific states at the start point of Grover's search, it is possible to reduce the search space of Grover's search for more efficient search. When the uniform superposition of specific states is used for the start of Grover's algorithm, the algorithm is called as the _amplitude amplification_. 

### State preparation
A `state_preparation` argument is used to specify a quantum circuit that prepares a quantum state for the start point of the amplitude amplification.
By default, a circuit with $H^{\otimes n} $ is used to prepare uniform superposition (so it will be Grover's serach). The diffusion circuit of the amplitude amplification relrects `state_preparation` automatically.

In [6]:
import numpy as np

# Specifying `state_preparation` 
# to preparet a superposition of |01>, |10>, and |11>
oracle = QuantumCircuit(3)
oracle.h(2)
oracle.ccx(0,1,2)
oracle.h(2)

theta = 2 * np.arccos(1 / np.sqrt(3))
state_preparation = QuantumCircuit(3)
state_preparation.ry(theta, 0)
state_preparation.ch(0,1)
state_preparation.x(1)
state_preparation.h(2)
grover = Grover(oracle=oracle, state_preparation=state_preparation,good_state=['11'])
# state_preparation
print(grover._grover_operator.state_preparation)


     ┌────────────┐          
q_0: ┤ RY(1.9106) ├──■───────
     └────────────┘┌─┴─┐┌───┐
q_1: ──────────────┤ H ├┤ X ├
         ┌───┐     └───┘└───┘
q_2: ────┤ H ├───────────────
         └───┘               


In [7]:
# Grover operator
print(grover._grover_operator)


                             ┌─────────────┐┌───┐     ┌───┐┌────────────┐     »
state_0: ───────■─────────■──┤ RY(-1.9106) ├┤ X ├──■──┤ X ├┤ RY(1.9106) ├──■──»
                │  ┌───┐┌─┴─┐└────┬───┬────┘└───┘  │  ├───┤└────────────┘┌─┴─┐»
state_1: ───────■──┤ X ├┤ H ├─────┤ X ├────────────■──┤ X ├──────────────┤ H ├»
         ┌───┐┌─┴─┐├───┤├───┤     ├───┤     ┌───┐┌─┴─┐├───┤    ┌───┐     ├───┤»
state_2: ┤ H ├┤ X ├┤ H ├┤ H ├─────┤ X ├─────┤ H ├┤ X ├┤ H ├────┤ X ├─────┤ H ├»
         └───┘└───┘└───┘└───┘     └───┘     └───┘└───┘└───┘    └───┘     └───┘»
«              
«state_0: ─────
«         ┌───┐
«state_1: ┤ X ├
«         └───┘
«state_2: ─────
«              


Fore more complicated oracles, state preparation, and options (e.g. `mcx_mode`), we can also pass `GroverOperator` to `Grover`.

In [8]:
from qiskit.circuit.library import GroverOperator
oracle = QuantumCircuit(2)
oracle.cz(0,1)
state_preparation = QuantumCircuit(2)
theta = 2 * np.arccos(1 / np.sqrt(3))
state_preparation = QuantumCircuit(2)
state_preparation.ry(theta, 0)
state_preparation.ch(0,1)
state_preparation.x(1)
grover_op = GroverOperator(oracle=oracle, state_preparation=state_preparation, insert_barriers=True, mcx_mode='noancilla')

grover=Grover(oracle=grover_op.oracle, good_state= ['11'], grover_operator=grover_op)
print(grover._grover_operator)


             ░           ┌─────────────┐ ░ ┌───┐          ┌───┐      ░ »
state_0: ─■──░────────■──┤ RY(-1.9106) ├─░─┤ X ├───────■──┤ X ├──────░─»
          │  ░ ┌───┐┌─┴─┐└─────────────┘ ░ ├───┤┌───┐┌─┴─┐├───┤┌───┐ ░ »
state_1: ─■──░─┤ X ├┤ H ├────────────────░─┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├─░─»
             ░ └───┘└───┘                ░ └───┘└───┘└───┘└───┘└───┘ ░ »
«         ┌────────────┐          
«state_0: ┤ RY(1.9106) ├──■───────
«         └────────────┘┌─┴─┐┌───┐
«state_1: ──────────────┤ H ├┤ X ├
«                       └───┘└───┘


## Dive into other arguments of `Grover`
`Grover` has arugments other than `oracle` and `state_preparation`. We will explain them in this section.

### Specifying `good_state`
`good_state` is used to check whether the measument result is correct or not internally. It can be a list of binary strings, a list of integer, `Statevector`, and Callable. If the input is a list of bitstrings, each bitstrings in the list represents a good state. If the input is a list of integer, each integer represent the index of the good state to be $|1\rangle$. If it is a `Statevector`, it represents a superposition of all good states.


In [9]:
# a list of binary strings good state
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
good_state = ['11', '00']
grover = Grover(oracle=oracle, good_state=good_state)
print(grover.is_good_state('11'))


True


In [10]:
# a list of integer good state
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
good_state = [0, 1]
grover = Grover(oracle=oracle, good_state=good_state)
print(grover.is_good_state('11'))

True


In [11]:
from qiskit.quantum_info import Statevector

# `Statevector` good state
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
good_state = Statevector.from_label('11')
grover = Grover(oracle=oracle, good_state=good_state)
print(grover.is_good_state('11'))

True


In [12]:
# Callable good state
def callable_good_state(bitstr):
    if bitstr == "11":
        return True
    else:
        return False

oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=callable_good_state)
print(grover.is_good_state('11'))

True


### The number of `iterations`
The number of repetition of applying the grover operator is important to obtain the correct result with Grover's algortihm. The number of iteration can be set by the `iteration` argument of `Grover`. `iteration` can be an integer or a list of integers. If `iteration` is an integer, `Grover` repeat applying its grover operator the number of `iteration` times. If `iteration` is a list of integer, `Grover` repeat applying its grover operator the number of `iteration` times in order. When `sample_from_iterations` is `True`, a randome integer between 0 and the specified number in `iteration` is used as the repetation number of grover iteration. This approach is useful when we don't even know the number of solution.
For more details of the algorithm using `sample_from_iterations`, see "Boyer et al., Tight bounds on quantum searching [arxiv:quant-ph/9605034](https://arxiv.org/abs/quant-ph/9605034)"

In [13]:
# integer iteration
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=['11'], iterations=1)


In [14]:
# list iteration
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=['11'], iterations=[1, 2, 3])

In [15]:
# using sample_from_iterations
oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=['11'], iterations=[1, 2, 3], sample_from_iterations=True)

When the number of solutions is known, we can also use a static method `optimal_num_iterations` to find the optimal number of iterations. Note that the output iterations is an approximate value. When the number of qubits is small, the output iterations may not be optimal.


In [16]:
iterations = Grover.optimal_num_iterations(num_solutions=1, num_qubits=8)
iterations

12

### Applying `post_processing`
We can apply an optional post processing to the top measurement for ease of readability. It can be used e.g. to convert from the bit-representation of the measurement `[1, 0, 1]` to a DIMACS CNF format `[1, -2, 3]`.

In [17]:
def to_DIAMACS_CNF_format(bit_rep):
    return [index+1 if val==1 else -1 * (index + 1) for index, val in enumerate(bit_rep)]

oracle = QuantumCircuit(2)
oracle.cz(0, 1)
grover = Grover(oracle=oracle, good_state=['11'],post_processing=to_DIAMACS_CNF_format)
grover.post_processing([1, 0, 1])

[1, -2, 3]